![MSE Logo](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png)

# AdvNLP Lab 2: Testing a pretrained word2vec model on analogy tasks

**Objectives:**  experiment with *word vectors* from word2vec: test them on analogy tasks; use *accuracy and MRR* scores.

**Useful documentation:** the [section on KeyedVectors in Gensim](https://radimrehurek.com/gensim/models/keyedvectors.html) and possibly the [section on word2vec](https://radimrehurek.com/gensim/models/word2vec.html).

## 1. Word2vec model trained on Google News
**1a.** Please install the latest version of Gensim, preferably in a Conda environment. 

In [1]:
#!pip install --upgrade gensim
# You can run the following verification:
!pip show gensim

Name: gensim
Version: 4.3.3
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: C:\Users\a.cavalli\AppData\Local\anaconda3\envs\AdvNLP\Lib\site-packages
Requires: numpy, scipy, smart-open
Required-by: 


In [2]:
import gensim, os, random
from gensim import downloader
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim import utils
# help(gensim.models.word2vec) # take a look if needed
import time
import itertools

**1b.** Please download from Gensim the `word2vec-google-news-300` model, upon your first use.  Then, please write code to answer the following questions:
* Where is the model stored on your computer and what is the file name?  You can store the absolute path in a variable called `path_to_model_file`.
* What is the size of the corresponding file?  Please display the size in gigabytes with two decimals.

In [3]:
# Download the model from Gensim (needed only the first time)
#gensim.downloader.load("word2vec-google-news-300")
# No need to store the returned value (uses a lot of memory).

In [4]:
# Please write your Python code below and execute it.
path_to_model_file = gensim.downloader.load("word2vec-google-news-300", return_path=True)
print(path_to_model_file)
size_gb = os.path.getsize(path_to_model_file) / 2**30
print('file size : %.2f GB' % size_gb)

C:\Users\a.cavalli/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
file size : 1.62 GB


**1c.** Please load the word2vec model as an instance of the class `KeyedVectors`, and store it in a variable called `wv_model`. 
What is, at this point, the memory size of the process corresponding to this notebook?  Simply write the value you obtain from any OS-specific utility that you like.

In [5]:
# Please write your Python code below and execute it.  Write the memory size on a commented line.
wv_model = KeyedVectors.load_word2vec_format(path_to_model_file, binary=True)

Memory size of the python process: 4152 MB.

**1d.** Please write the instructions that generate the answers to the following questions.
* What is the size of the vocabulary of the `wv_model` model?  
* What is the dimensionality of each word vector?  
* What is the word corresponding to the vector in position 1234?  
* What are the first 10 coefficients of the word vector for the word *pyramid*?  

In [6]:
# Please write your Python code below and execute it.
print('Vocabulary size:', len(wv_model.key_to_index))
print('Vector shape:', wv_model.get_vector('test').shape)
print('Word in position 1234:', wv_model.index_to_key[1234])
print('"pyramid"\'s first 10 coefficients:', wv_model.get_vector('pyramid')[:10])

Vocabulary size: 3000000
Vector shape: (300,)
Word in position 1234: learn
"pyramid"'s first 10 coefficients: [ 0.00402832 -0.00260925  0.04296875  0.19433594 -0.03979492 -0.06445312
  0.42773438 -0.18359375 -0.27148438 -0.12890625]


## 2. Solving analogies using word2vec trained on Google News
In this section, you are going to use word vectors to solve analogy tasks provided with Gensim, such as "What is to France what Rome is to Italy?".  The predefined function in Gensim that evaluates a model on this task does not provide enough details, so you will need to make modifications to it.

**2a.** The analogy tasks are stored in a text file called `questions-words.txt` which is typically found in `C:\Users\YourNameHere\.conda\envs\YourEnvNameHere\Lib\site-packages\gensim\test\test_data`.  You can access it from here with Gensim as `datapath('questions-words.txt')`.  

Please create a file called `questions-words-100.txt` with the first 100 lines from the original file.  Please run the evaluation task on this file, using the [documentation of the KeyedVectors class](https://radimrehurek.com/gensim/models/keyedvectors.html), then answer the following questions:
* How many analogy tasks are there in your `questions-words-100.txt` file?
* How many analogies were solved correctly and how many incorrectly?
* What is the accuracy returned by `evaluate_word_analogies`?
* How much time did it take to solve the analogies?

In [17]:
# Please write your Python code below and execute it.
import time

start = time.time()
(overall_score, sections) = wv_model.evaluate_word_analogies('questions-words-100.txt')
end = time.time()
duration = end - start

In [18]:
print(f'execution time: {duration} seconds')
print('overall accuracy:', overall_score)
for section in sections:
    print('section', section['section'])
    print('- correct:', len(section['correct']))
    print('- incorrect:', len(section['incorrect']))

execution time: 2.2704060077667236 seconds
overall accuracy: 0.8080808080808081
section capital-common-countries
- correct: 80
- incorrect: 19
section Total accuracy
- correct: 80
- incorrect: 19


The file contains 99 analogy tasks

**2b.** Please answer in writing the following questions:
* What is the meaning of the first line of `questions-words-100.txt`?
* How many analogies are there in the original `questions-words.txt`?
* How much time would it take to solve the original set of analogies?

In [19]:
analogies_count_all = 0
with open(datapath('questions-words.txt')) as file:
    for line in file:
        if not line.startswith(':'):
            analogies_count_all += 1

print('total number of analogies:', analogies_count_all)

analogies_count_subset = 99 # subset of questions-words-100.txt
expected_duration_all = duration / analogies_count_subset * analogies_count_all
print('expected duration for all analogies:', expected_duration_all)

total number of analogies: 19544
expected duration for all analogies: 448.2102526847762


- The first line represents the beginning of a section named "capital-common-countries"
- The original `questions-words.txt` file contains 19544 analogies
- It would take around 445 seconds (around 7 minutes) to execute the original set of analogies. Note: the first execution of the "questions-words-100" dataset takes longer ; the expected duration is computed on the duration of the second execution.

**2c.** The built-in function from Gensim has several weaknesses, which you will address here.  Please copy the source code of the function `evaluate_word_analogies` from the file `gensim\models\keyedvectors.py` and create here a new function which will improve the built-in one as follows.  The function will be called `my_evaluate_word_analogies` and you will also pass it the model as the first argument.  Overall, please proceed gradually and only make minimal modifications, to ensure you don't break the function.  It is important to first understand the structure of the result, `analogies_scores` and `sections`. 

* Modify the line where `section[incorrect]` is assembled in order to also add to each analogy the *incorrect guess* (i.e. what the model thought was the good answer, but got it wrong).

* Modify the code so that when `section[incorrect]` is assembled, you also add the *rank of the correct answer* among the candidates returned by the system (after the incorrect guess).  If the correct answer is not present at all, then code the rank as 0.

In [10]:
def my_evaluate_word_analogies(model, analogies, restrict_vocab=300000, case_insensitive=True, dummy4unknown=False, similarity_function='most_similar'):
    logger = gensim.logger
    ok_keys = model.index_to_key[:restrict_vocab]
    if case_insensitive:
        ok_vocab = {k.upper(): model.get_index(k) for k in reversed(ok_keys)}
    else:
        ok_vocab = {k: model.get_index(k) for k in reversed(ok_keys)}
    oov = 0
    logger.info("Evaluating word analogies for top %i words in the model on %s", restrict_vocab, analogies)
    sections, section = [], None
    quadruplets_no = 0
    with utils.open(analogies, 'rb') as fin:
        for line_no, line in enumerate(fin):
            line = utils.to_unicode(line)
            if line.startswith(': '):
                # a new section starts => store the old section
                if section:
                    sections.append(section)
                    model._log_evaluate_word_analogies(section)
                section = {'section': line.lstrip(': ').strip(), 'correct': [], 'incorrect': []}
            else:
                if not section:
                    raise ValueError("Missing section header before line #%i in %s" % (line_no, analogies))
                try:
                    if case_insensitive:
                        a, b, c, expected = [word.upper() for word in line.split()]
                    else:
                        a, b, c, expected = [word for word in line.split()]
                except ValueError:
                    logger.info("Skipping invalid line #%i in %s", line_no, analogies)
                    continue
                quadruplets_no += 1
                if a not in ok_vocab or b not in ok_vocab or c not in ok_vocab or expected not in ok_vocab:
                    oov += 1
                    if dummy4unknown:
                        logger.debug('Zero accuracy for line #%d with OOV words: %s', line_no, line.strip())
                        section['incorrect'].append((a, b, c, expected))
                    else:
                        logger.debug("Skipping line #%i with OOV words: %s", line_no, line.strip())
                    continue
                original_key_to_index = model.key_to_index
                model.key_to_index = ok_vocab
                ignore = {a, b, c}  # input words to be ignored
                predicted = None
                guess_count = 0
                right_prediction_rank = 0
                
                # find the most likely prediction using 3CosAdd (vector offset) method
                # TODO: implement 3CosMul and set-based methods for solving analogies
    
                sims = model.most_similar(positive=[b, c], negative=[a], topn=5, restrict_vocab=restrict_vocab)
                model.key_to_index = original_key_to_index
                for element in sims:
                    current = element[0].upper() if case_insensitive else element[0]
                    if current not in ok_vocab or current in ignore:
                        continue
                        
                    guess_count += 1 # count only valid similarities
                    if predicted is None:
                        predicted = current
                        if predicted == expected:
                            break # 1st prediction is correct
                        else:
                            logger.debug("%s: expected %s, predicted %s", line.strip(), expected, predicted)
                    elif current == expected:
                        right_prediction_rank = guess_count
                        logger.debug("%s: expected %s, guessed right at rank %i", line.strip(), expected, right_prediction_rank)
                        break # 1st prediction wasn't correct, but the n-th prediction is correct (see right_prediction_rank)
                        
                if predicted == expected:
                    section['correct'].append((a, b, c, expected))
                else:
                    section['incorrect'].append((a, b, c, expected, predicted, right_prediction_rank))
    if section:
        # store the last section, too
        sections.append(section)
        model._log_evaluate_word_analogies(section)
    
    total = {
        'section': 'Total accuracy',
        'correct': list(itertools.chain.from_iterable(s['correct'] for s in sections)),
        'incorrect': list(itertools.chain.from_iterable(s['incorrect'] for s in sections)),
    }
    
    oov_ratio = float(oov) / quadruplets_no * 100
    logger.info('Quadruplets with out-of-vocabulary words: %.1f%%', oov_ratio)
    if not dummy4unknown:
        logger.info(
            'NB: analogies containing OOV words were skipped from evaluation! '
            'To change this behavior, use "dummy4unknown=True"'
        )
    analogies_score = model._log_evaluate_word_analogies(total)
    sections.append(total)
    # Return the overall score and the full lists of correct and incorrect analogies
    return analogies_score, sections

**2d.** Please run the `my_evaluate_word_analogies` function on `questions-words-100.txt` and then write instructions to display, from the results stored in `analogy_scores`:
* one incorrectly-solved analogy (selected at random), including also the error made by the model and the rank of the correct answer, thus adding:
  - a fifth word, which is the incorrect one found by the model
  - a sixth term, which is the integer indicating the rank (or 0)
* one correctly-solved analogy selected at random (in principle, four terms).

In [11]:
# Please write your Python code below and execute it.
analogy_scores = my_evaluate_word_analogies(wv_model, 'questions-words-100.txt')

In [12]:
(overall_score, sections) = analogy_scores
section = sections[0]

print('random incorrect analogy:', random.choice(section['incorrect']))
print('random correct analogy:', random.choice(section['correct']))

random incorrect analogy: ('BAGHDAD', 'IRAQ', 'CANBERRA', 'AUSTRALIA', 'MR_RUDD', 2)
random correct analogy: ('ATHENS', 'GREECE', 'BERLIN', 'GERMANY')


**2e.** Please write a function to compute the MRR score given a structure with correctly and incorrectly solved analogies, such as the one that is found in the results from `evaluate_word_analogies`.  The structure is not divided into categories.

The Mean Reciprocal Rank (please use the [formula here](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)) gives some credit for incorrectly solved analogies, in inverse proportion to the rank of the correct answer among the candidates.  This rank is 1 for correctly solved analogies (full credit), and 1/k (or 0) for incorrectly solved ones.

In [13]:
# Please define here the function that computes MRR from the information stored in analogy_scores
def myMRR(analogies):
    correct_count = len(analogies['correct'])
    incorrect_count = len(analogies['incorrect'])
    total_count = correct_count + incorrect_count
    
    mrr = correct_count
    for incorrect in analogies['incorrect']:
        rank = incorrect[-1]
        if rank > 0:
            mrr += 1 / rank
    return mrr / total_count

In [14]:
# Please test your MRR function by running the following code, which  displays the total number of analogy tasks, 
# the number of different categories (sections), the accuracy of the results (total number of correctly 
# solved analogies), and the MRR score of the results:
print("Total number of analogies:",  # The last dictionary is the total
      len(analogy_scores[1][-1]['correct']) + 
      len(analogy_scores[1][-1]['incorrect']))
print("Total number of categories:", len(analogy_scores[1]) - 1) # the "total" is excluded 
print(f"Overall accuracy: {analogy_scores[0]:.2f} and MRR: {myMRR(analogy_scores[1][-1]):.2f}")

Total number of analogies: 99
Total number of categories: 1
Overall accuracy: 0.81 and MRR: 0.86


**2f.** When you have some time, please compute the accuracy and MRR and the total time for the entire `questions-words.txt` file.  Is the timing compatible with your estimate from (2b)?  What do you think about the difference between accuracy and MRR? 

In [20]:
# Please write your Python code below and execute it.
start = time.time()
analogy_scores = my_evaluate_word_analogies(wv_model, datapath('questions-words.txt'))
end = time.time()
duration = end - start
print(f'execution time: {duration} seconds')

execution time: 401.6764090061188 seconds


The execution time (401 seconds) is close to the expected one (448 seconds).

In [21]:
print("Total number of analogies:",  # The last dictionary is the total
      len(analogy_scores[1][-1]['correct']) + 
      len(analogy_scores[1][-1]['incorrect']))
print("Total number of categories:", len(analogy_scores[1]) - 1) # the "total" is excluded 
print(f"Overall accuracy: {analogy_scores[0]:.2f} and MRR: {myMRR(analogy_scores[1][-1]):.2f}")

Total number of analogies: 19330
Total number of categories: 14
Overall accuracy: 0.74 and MRR: 0.79


For the accuracy of one task, we only considers whether the task was performed correctly: a correct analogy gets a score of 1 and and incorrect analogy gets a score of 0. The overall accuracy is the sum of the individual scores divided by the number of tasks.

For an incorrect analogy, the MRR score also considers the other results proposed by the model (from rank number 2). The MRR score for a task gives a score of 1 for a correct analogy (as for the accuracy), and a score between 0 and 0.5 for an incorrect analogy, based on the rank of the correct analogy among the proposed results. As for the overall accuracy, the overall MRR score is the sum of the individual scores divided by the number of tasks.

Thus, the minimum MRR score we can get is equal to the accuracy. We get this minimum score when, among all proposed results of an incorrect analogy, the model never proposed the correct one.
If the model proposes a correct result for at least one incorrect analogy, the MRR score will be greater than the accuracy.
For a given accuracy, the maximum MRR score a model can get is `accuracy + (1 - accuracy) * 0.5`. This MRR score can be reached when, for all incorrect analogies, the correct result was the second proposed result.

## End of AdvNLP Lab 2
Please make sure all cells have been executed, save this completed notebook, and upload it to Moodle.